In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import requests
from datetime import datetime
from functools import reduce

# DB 연동

In [2]:
import cx_Oracle as cx
conn = cx.connect("hr","hr","127.0.01:1521/XE") #DB연동
cur = conn.cursor()
cur.execute("select * from employees") #sql 쿼리 작성


<cx_Oracle.Cursor on <cx_Oracle.Connection to hr@127.0.01:1521/XE>>

In [3]:
def usd_kwr():
    url = Request('https://kr.investing.com/currencies/usd-krw', headers={'user-agent': 'Mozilla/5.0 (Windows NT 11.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.76 Safari/537.36 NetHelper70'})

    bs = BeautifulSoup(urlopen(url), 'html.parser')

    usd_kwr= float(bs.select_one('#__next > div.desktop\:relative.desktop\:bg-background-default > div > div > div.grid.gap-4.tablet\:gap-6.grid-cols-4.tablet\:grid-cols-8.desktop\:grid-cols-12.grid-container--fixed-desktop.general-layout_main__LHpKS > main > div > div.instrument-header_instrument-header__i63mN.mb-5.bg-background-surface.tablet\:grid.tablet\:grid-cols-2 > div:nth-child(2) > div.instrument-price_instrument-price__2w9MW.flex.items-end.flex-wrap.font-bold > span').text.replace(',',''))
    
    return usd_kwr


In [5]:
def dollar_index():
    url = Request('https://kr.investing.com/currencies/us-dollar-index', headers={'user-agent': 'Mozilla/5.0 (Windows NT 11.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.76 Safari/537.36 NetHelper70'})

    bs = BeautifulSoup(urlopen(url), 'html.parser')
    dollar_index = float(bs.select_one('#last_last').text.replace(',',''))

    return dollar_index
    

In [6]:
def cny_krw():
    url = Request('https://kr.investing.com/currencies/cny-krw', headers={'user-agent': 'Mozilla/5.0 (Windows NT 11.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.76 Safari/537.36 NetHelper70'})

    bs = BeautifulSoup(urlopen(url), 'html.parser')
    cny_krw = float(bs.select_one('#__next > div.desktop\:relative.desktop\:bg-background-default > div > div > div.grid.gap-4.tablet\:gap-6.grid-cols-4.tablet\:grid-cols-8.desktop\:grid-cols-12.grid-container--fixed-desktop.general-layout_main__LHpKS > main > div > div.instrument-header_instrument-header__i63mN.mb-5.bg-background-surface.tablet\:grid.tablet\:grid-cols-2 > div:nth-child(2) > div.instrument-price_instrument-price__2w9MW.flex.items-end.flex-wrap.font-bold > span').text.replace(',',''))
    
    return cny_krw

In [7]:
def eur_krw():
    url = Request('https://kr.investing.com/currencies/eur-krw', headers={'user-agent': 'Mozilla/5.0 (Windows NT 11.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.76 Safari/537.36 NetHelper70'})

    bs = BeautifulSoup(urlopen(url), 'html.parser')
    eur_krw = float(bs.select_one('#__next > div.desktop\:relative.desktop\:bg-background-default > div > div > div.grid.gap-4.tablet\:gap-6.grid-cols-4.tablet\:grid-cols-8.desktop\:grid-cols-12.grid-container--fixed-desktop.general-layout_main__LHpKS > main > div > div.instrument-header_instrument-header__i63mN.mb-5.bg-background-surface.tablet\:grid.tablet\:grid-cols-2 > div:nth-child(2) > div.instrument-price_instrument-price__2w9MW.flex.items-end.flex-wrap.font-bold > span').text.replace(',',''))
    
    return eur_krw

In [8]:
def jpy_krw():
    url = Request('https://kr.investing.com/currencies/jpy-krw', headers={'user-agent': 'Mozilla/5.0 (Windows NT 11.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.5414.76 Safari/537.36 NetHelper70'})

    bs = BeautifulSoup(urlopen(url), 'html.parser')
    jpy_krw = float(bs.select_one('#__next > div.desktop\:relative.desktop\:bg-background-default > div > div > div.grid.gap-4.tablet\:gap-6.grid-cols-4.tablet\:grid-cols-8.desktop\:grid-cols-12.grid-container--fixed-desktop.general-layout_main__LHpKS > main > div > div.instrument-header_instrument-header__i63mN.mb-5.bg-background-surface.tablet\:grid.tablet\:grid-cols-2 > div:nth-child(2) > div.instrument-price_instrument-price__2w9MW.flex.items-end.flex-wrap.font-bold > span').text.replace(',',''))
    
    return jpy_krw

In [9]:
def job():
#     dollar_index_time, kospi_200_time, spx_500_time = '00:00:00','00:00:00','00:00:00'
#     usd_kwr,  dollar_index, kospi_200,spx_500 = '0','0','0','0'
    return usd_kwr(),  dollar_index(), cny_krw(), eur_krw(), jpy_krw()

In [ ]:
import urllib.error
import urllib.request
import time
from datetime import datetime

while True:
    try:
        current_time = datetime.now()
        usd_kwr_value, dollar_index_value, cny_krw_value, eur_krw_value, jpy_krw_value = job()
        cur.execute("INSERT INTO EXRate (rate_time, usd, cny, jpy, euro, dollar_index) VALUES "
                    "(:rate_time, :usd, :cny, :jpy, :euro, :dollar_index)",
                    {'rate_time': str(current_time)[:17] + '00',
                     'usd': usd_kwr_value,
                     'cny': cny_krw_value,
                     'jpy': jpy_krw_value,
                     'euro': eur_krw_value,
                     'dollar_index': dollar_index_value})
        cur.execute("commit")

    except Exception as e:
        # Catch any exceptions that might occur during the job() function or database operations.
        # Instead of handling the exception here, we just pass over it silently.
        pass

    finally:
        time.sleep(55)
